In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import operator
import time, datetime
from utils import *

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# modified 0211

In [2]:
bath_path = '/mnt/qb/work/mlcolab/hzhou52/kt/Duolingo/learning_traces.csv'
log = pd.read_csv(bath_path, encoding = "utf-8",low_memory=False)

In [9]:
log.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct
0,1.0,1362076081,27649635,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,lernt/lernen<vblex><pri><p3><sg>,6,4,2,2
1,0.5,1362076081,27649635,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,die/die<det><def><f><sg><nom>,4,4,2,1
2,1.0,1362076081,27649635,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,mann/mann<n><m><sg><nom>,5,4,1,1
3,0.5,1362076081,27649635,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,frau/frau<n><f><sg><nom>,6,5,2,1
4,1.0,1362076081,27649635,u:FO,de,en,84920990d78044db53c1b012f5bf9ab5,das/das<det><def><nt><sg><nom>,4,4,1,1


In [10]:
log.columns

Index(['p_recall', 'timestamp', 'delta', 'user_id', 'learning_language',
       'ui_language', 'lexeme_id', 'lexeme_string', 'history_seen',
       'history_correct', 'session_seen', 'session_correct'],
      dtype='object')

In [13]:
check_nan(log)
count_unique(log, ['user_id', 'learning_language', 'lexeme_id'])
print('total number of logs: ', len(log))

number of NaN in column p_recall: 0
number of NaN in column timestamp: 0
number of NaN in column delta: 0
number of NaN in column user_id: 0
number of NaN in column learning_language: 0
number of NaN in column ui_language: 0
number of NaN in column lexeme_id: 0
number of NaN in column lexeme_string: 0
number of NaN in column history_seen: 0
number of NaN in column history_correct: 0
number of NaN in column session_seen: 0
number of NaN in column session_correct: 0
number of unique value in user_id: 115222
number of unique value in learning_language: 6
number of unique value in lexeme_id: 19279
total number of logs:  12854226


In [15]:
user1 = log.loc[log.user_id == 'u:FO']
len(user1.lexeme_id.unique())

12

In [17]:
user1

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct
0,1.000000,1362076081,27649635,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,lernt/lernen<vblex><pri><p3><sg>,6,4,2,2
1,0.500000,1362076081,27649635,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,die/die<det><def><f><sg><nom>,4,4,2,1
2,1.000000,1362076081,27649635,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,mann/mann<n><m><sg><nom>,5,4,1,1
3,0.500000,1362076081,27649635,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,frau/frau<n><f><sg><nom>,6,5,2,1
4,1.000000,1362076081,27649635,u:FO,de,en,84920990d78044db53c1b012f5bf9ab5,das/das<det><def><nt><sg><nom>,4,4,1,1
5,1.000000,1362076081,27649635,u:FO,de,en,56429751fdaedb6e491f4795c770f5a4,der/der<det><def><m><sg><nom>,4,3,1,1
6,1.000000,1362076081,27649635,u:FO,de,en,1bacf218eaaf9f944e525f7be9b31899,kind/kind<n><nt><sg><nom>,4,4,1,1
8,1.000000,1362082044,5963,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,lernt/lernen<vblex><pri><p3><sg>,8,6,6,6
9,0.750000,1362082044,5963,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,die/die<det><def><f><sg><nom>,6,5,4,3
10,0.888889,1362082044,5963,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,mann/mann<n><m><sg><nom>,6,5,9,8


In [16]:
user1.lexeme_id.unique()

array(['76390c1350a8dac31186187e2fe1e178',
       '7dfd7086f3671685e2cf1c1da72796d7',
       '35a54c25a2cda8127343f6a82e6f6b7d',
       '0cf63ffe3dda158bc3dbd55682b355ae',
       '84920990d78044db53c1b012f5bf9ab5',
       '56429751fdaedb6e491f4795c770f5a4',
       '1bacf218eaaf9f944e525f7be9b31899',
       '4fcb6bb8e44d7b618999721071862827',
       'a6834806c43ea1be9eb3e4fdae6f98db',
       'dd34978165d17f7e729a2ef331a7600d',
       '495f763ef6027e020c53431484aa5ede',
       '46b112cd07fdcc98db8670a5d71c613d'], dtype=object)